# NBA Player Stats Scraper
#### Scrapes the roster table from 
#### https://www.espn.com/nba/stats/player/_/table/offensive/sort/avgPoints/dir/desc 
#### and saves data to csv file

In [1]:
# Imports
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import csv

##### Helper methods

In [2]:
def get_game_stats(url):
    # Get json response from url
    player_page = requests.get(url)
    # Put response into beautifulsoup
    s = BeautifulSoup(player_page.text, 'lxml')
    # Find the Regular Season Averages table on the page
    stats_table = s.find('section', class_='ResponsiveTable ResponsiveTable--fixed-left pt4')
    # Find the left side subtable containing SEASON and TEAM
    left_table = stats_table.find('table', class_='Table Table--align-right Table--fixed Table--fixed-left')
    # Get left table body
    left_table = left_table.find('tbody')
    # Process the left table
#     print(process_left_table(left_table))
    left_stats = process_left_table(left_table)
    
    # Get right table body
    right_table = stats_table.find('div', class_='Table__ScrollerWrapper relative overflow-hidden').find('tbody', class_='Table__TBODY')
#     print(right_table)
    # Process the right table
    right_stats = process_right_table(right_table)

    # join results from left and right tables
    season = 0
    results = []
    for season in range(len(right_stats)):
        results.append(left_stats[season] + right_stats[season])
#     print(results)
    return results    

In [3]:
def process_left_table(tbody):
    count = 0
    season = 0
    results = []
    for tag in tbody:
        for subtag in tag:
            count += 1
            if ((subtag.text == 'Career') or (subtag.text == '')):
                continue
            if (count == 2):
                results[season].append(subtag.text)
                count = 0
                season += 1
            else:
                results.append([subtag.text])
    return results

In [4]:
def process_right_table(tbody):
    count = 0
    season = 0
    results = []
    for row in tbody:
        for stat in row:
            if (count == 0):
                results.append([stat.text])
            elif (count == 17):
                results[season].append(stat.text)
                count = 0
                season += 1
                continue
            else:
                results[season].append(stat.text)
            count += 1
#     print(results[:-1])
    # Last element contains career averages, which we don't need
    return results[:-1]

#### Main routine

In [5]:
# csv out file name
foutName = 'nba_stats.csv'
# csv file to write cleaned data into
fout = open(foutName, 'w')
# csv writer to write to csv file
foutWriter = csv.writer(fout)

# Write header to csv
header = ['LASTNAME', 'FIRSTNAME', 'SEASON', 'TEAM', 'NUMBER', 'POS', 'AGE', 'HEIGHT', 'WEIGHT', 'COLLEGE', 'SALARY', 'GP', 'GS', 'MIN', 'FG', 'FG%', '3PT', '3P%', 'FT', 'FT%', 'OR', 'DR', 'REB', 'AST', 'BLK', 'STL', 'PF', 'TO', 'PTS']
header_line = ''
for key in header:
    header_line += key + ','
fout.write(header_line[:-1] + '\n')

# Team to process
team_name = ['atl/atlanta-hawks', 'bos/boston-celtics', 'bkn/brooklyn-nets', 'cha/charlotte-hornets', 'chi/chicago-bulls', 'cle/cleveland-cavaliers', 'dal/dallas-mavericks', 'den/denver-nuggets', 'det/detroit-pistons', 'gs/golden-state-warriors', 'hou/houston-rockets', 'ind/indiana-pacers', 'lac/la-clippers', 'lal/los-angeles-lakers', 'mem/memphis-grizzlies', 'mia/miami-heat', 'mil/milwaukee-bucks', 'min/minnesota-timberwolves', 'no/new-orleans-pelicans', 'ny/new-york-knicks', 'okc/oklahoma-city-thunder', 'orl/orlando-magic', 'phi/philadelphia-76ers', 'phx/phoenix-suns', 'por/portland-trail-blazers', 'sac/sacramento-kings', 'sa/san-antonio-spurs', 'tor/toronto-raptors', 'utah/utah-jazz', 'wsh/washington-wizards']

In [6]:
for team in team_name:
    # url where team roster table is located
    url = 'https://www.espn.com/nba/team/roster/_/name/' + team
    # print(url)

    # Get json response from url
    page = requests.get(url)
    # Put response into beautifulsoup
    soup = BeautifulSoup(page.text, 'lxml')
    # Find where all player tags are burried
    table_tag = soup.find('tbody')
    # table_tag

    for player_row_tag in table_tag:
        try:
            # Working on one player
            count = 0
            row_info = []
            for col_tag in player_row_tag:
                # Get the player's url
                if count == 0:
                    player_url_tag = col_tag.find('a', href = re.compile(r'[/]([a-z]|[A-Z])\w+')).attrs['href']
                    row_info.append(player_url_tag)
                # Get player name & number
                if count == 1:
                    col1_tag = col_tag.find('span')
                    for subtag in col1_tag:
                        row_info.append(subtag.text)
                else:
                    # Get everything else for this player in the roster table
                    row_info.append(col_tag.text)
                count += 1

            try:
                # Go to the player's url page to get game stats
                season_stats = get_game_stats(row_info[0][:31] + 'stats/' + row_info[0][31:])
#                 print(season_stats)

                # All player stats obtained. Print player's info to csv row
                url_ext = row_info[0]

                firstName = row_info[2].split()[0]
                lName = row_info[2].split()[1:]
                lastName = ''
                for name in lName:
                    lastName += name
                    if (name != lName[len(lName)-1]):
                        lastName += ' '
                number = row_info[3]
                pos = row_info[4]
                age = row_info[5]
                height = row_info[6][:1] + '-' + row_info[6][3:-1]
                weight = row_info[7][:3]
                college = row_info[8]
                if college == '--':
                    college = 'NaN'
                salary = row_info[9]
                salary = re.sub('[$,]', '', salary)

                # Print a row for each season the player has played
                for season in season_stats:
                    line = ''
                    if (season != season_stats[0]):
                        salary = 0
                    line = '{},{},{},{},{},{},{},{},{},{},{},'.format(lastName, firstName, season[0], season[1], number, pos, age, height, weight, college, salary)
                    # Add the player's season stats to the line
                    for stat in season[2:]:
                        if ((stat == season[5]) or (stat == season[7]) or (stat == season[9])):
                            stat = stat.split('-')[1]
                        line += stat
                        if (stat != season[len(season)-1]):
                            line += ','
                            
                    # PRINT TO STDOUT FOR CHECKING PROGRESS WHILE PROGRAM IS RUNNING
                    print(season[1], season[0], lastName, firstName)
                    
                    # PRINT TO CSV
                    fout.write(line + '\n')
            except:
                continue
        except:
            continue

ATL 2016-17 Bembry DeAndre'
ATL 2017-18 Bembry DeAndre'
ATL 2018-19 Bembry DeAndre'
ATL 2019-20 Bembry DeAndre'
ATL 2019-20 Brown Jr. Charlie
HOU 2014-15 Capela Clint
HOU 2015-16 Capela Clint
HOU 2016-17 Capela Clint
HOU 2017-18 Capela Clint
HOU 2018-19 Capela Clint
HOU 2019-20 Capela Clint
TOR 1998-99 Carter Vince
TOR 1999-00 Carter Vince
TOR 2000-01 Carter Vince
TOR 2001-02 Carter Vince
TOR 2002-03 Carter Vince
TOR 2003-04 Carter Vince
NJ 2004-05 Carter Vince
TOR 2004-05 Carter Vince
NJ 2005-06 Carter Vince
NJ 2006-07 Carter Vince
NJ 2007-08 Carter Vince
NJ 2008-09 Carter Vince
ORL 2009-10 Carter Vince
ORL 2010-11 Carter Vince
PHX 2010-11 Carter Vince
DAL 2011-12 Carter Vince
DAL 2012-13 Carter Vince
DAL 2013-14 Carter Vince
MEM 2014-15 Carter Vince
MEM 2015-16 Carter Vince
MEM 2016-17 Carter Vince
SAC 2017-18 Carter Vince
ATL 2018-19 Carter Vince
ATL 2019-20 Carter Vince
ATL 2017-18 Collins John
ATL 2018-19 Collins John
ATL 2019-20 Collins John
GS 2013-14 Dedmon Dewayne
ORL 2013-14 

MIN 2016-17 Dunn Kris
CHI 2017-18 Dunn Kris
CHI 2018-19 Dunn Kris
CHI 2019-20 Dunn Kris
CHI 2015-16 Felicio Cristiano
CHI 2016-17 Felicio Cristiano
CHI 2017-18 Felicio Cristiano
CHI 2018-19 Felicio Cristiano
CHI 2019-20 Felicio Cristiano
CHI 2019-20 Gafford Daniel
PHX 2017-18 Harrison Shaquille
CHI 2018-19 Harrison Shaquille
CHI 2019-20 Harrison Shaquille
CHI 2018-19 Hutchison Chandler
CHI 2019-20 Hutchison Chandler
NY 2017-18 Kornet Luke
NY 2018-19 Kornet Luke
CHI 2019-20 Kornet Luke
MIN 2014-15 LaVine Zach
MIN 2015-16 LaVine Zach
MIN 2016-17 LaVine Zach
CHI 2017-18 LaVine Zach
CHI 2018-19 LaVine Zach
CHI 2019-20 LaVine Zach
CHI 2017-18 Markkanen Lauri
CHI 2018-19 Markkanen Lauri
CHI 2019-20 Markkanen Lauri
CHI 2019-20 Mokoka Adam
WSH 2013-14 Porter Jr. Otto
WSH 2014-15 Porter Jr. Otto
WSH 2015-16 Porter Jr. Otto
WSH 2016-17 Porter Jr. Otto
WSH 2017-18 Porter Jr. Otto
CHI 2018-19 Porter Jr. Otto
WSH 2018-19 Porter Jr. Otto
CHI 2019-20 Porter Jr. Otto
WSH 2016-17 Satoransky Tomas
WSH 2

DET 2019-20 Bone Jordan
DET 2018-19 Brown Bruce
DET 2019-20 Brown Bruce
DET 2019-20 Doumbouya Sekou
NY 2014-15 Galloway Langston
NY 2015-16 Galloway Langston
NO 2016-17 Galloway Langston
SAC 2016-17 Galloway Langston
DET 2017-18 Galloway Langston
DET 2018-19 Galloway Langston
DET 2019-20 Galloway Langston
LAC 2010-11 Griffin Blake
LAC 2011-12 Griffin Blake
LAC 2012-13 Griffin Blake
LAC 2013-14 Griffin Blake
LAC 2014-15 Griffin Blake
LAC 2015-16 Griffin Blake
LAC 2016-17 Griffin Blake
DET 2017-18 Griffin Blake
LAC 2017-18 Griffin Blake
DET 2018-19 Griffin Blake
DET 2019-20 Griffin Blake
DET 2019-20 Hall Donta
MIL 2012-13 Henson John
MIL 2013-14 Henson John
MIL 2014-15 Henson John
MIL 2015-16 Henson John
MIL 2016-17 Henson John
MIL 2017-18 Henson John
MIL 2018-19 Henson John
CLE 2019-20 Henson John
DET 2019-20 Henson John
DET 2017-18 Kennard Luke
DET 2018-19 Kennard Luke
DET 2019-20 Kennard Luke
DET 2019-20 King Louis
DET 2011-12 Knight Brandon
DET 2012-13 Knight Brandon
MIL 2013-14 Knig

IND 2018-19 Johnson Alize
IND 2019-20 Johnson Alize
OKC 2012-13 Lamb Jeremy
OKC 2013-14 Lamb Jeremy
OKC 2014-15 Lamb Jeremy
CHA 2015-16 Lamb Jeremy
CHA 2016-17 Lamb Jeremy
CHA 2017-18 Lamb Jeremy
CHA 2018-19 Lamb Jeremy
IND 2019-20 Lamb Jeremy
IND 2017-18 Leaf T.J.
IND 2018-19 Leaf T.J.
IND 2019-20 Leaf T.J.
PHI 2015-16 McConnell T.J.
PHI 2016-17 McConnell T.J.
PHI 2017-18 McConnell T.J.
PHI 2018-19 McConnell T.J.
IND 2019-20 McConnell T.J.
CHI 2014-15 McDermott Doug
CHI 2015-16 McDermott Doug
CHI 2016-17 McDermott Doug
OKC 2016-17 McDermott Doug
DAL 2017-18 McDermott Doug
NY 2017-18 McDermott Doug
IND 2018-19 McDermott Doug
IND 2019-20 McDermott Doug
UTAH 2017-18 Mitrou-Long Naz
UTAH 2018-19 Mitrou-Long Naz
IND 2019-20 Mitrou-Long Naz
ORL 2013-14 Oladipo Victor
ORL 2014-15 Oladipo Victor
ORL 2015-16 Oladipo Victor
OKC 2016-17 Oladipo Victor
IND 2017-18 Oladipo Victor
IND 2018-19 Oladipo Victor
IND 2019-20 Oladipo Victor
OKC 2016-17 Sabonis Domantas
IND 2017-18 Sabonis Domantas
IND 201

UTAH 2018-19 Allen Grayson
MEM 2019-20 Allen Grayson
SA 2014-15 Anderson Kyle
SA 2015-16 Anderson Kyle
SA 2016-17 Anderson Kyle
SA 2017-18 Anderson Kyle
MEM 2018-19 Anderson Kyle
MEM 2019-20 Anderson Kyle
MEM 2017-18 Brooks Dillon
MEM 2018-19 Brooks Dillon
MEM 2019-20 Brooks Dillon
MEM 2019-20 Clarke Brandon
MIN 2013-14 Dieng Gorgui
MIN 2014-15 Dieng Gorgui
MIN 2015-16 Dieng Gorgui
MIN 2016-17 Dieng Gorgui
MIN 2017-18 Dieng Gorgui
MIN 2018-19 Dieng Gorgui
MIN 2019-20 Dieng Gorgui
MEM 2019-20 Dieng Gorgui
MEM 2019-20 Guduric Marko
PHX 2017-18 Jackson Josh
PHX 2018-19 Jackson Josh
MEM 2019-20 Jackson Josh
MEM 2018-19 Jackson Jr. Jaren
MEM 2019-20 Jackson Jr. Jaren
MIN 2015-16 Jones Tyus
MIN 2016-17 Jones Tyus
MIN 2017-18 Jones Tyus
MIN 2018-19 Jones Tyus
MEM 2019-20 Jones Tyus
MEM 2019-20 Konchar John
PHX 2018-19 Melton De'Anthony
MEM 2019-20 Melton De'Anthony
MEM 2019-20 Morant Ja
SA 2008-09 Tolliver Anthony
GS 2009-10 Tolliver Anthony
POR 2009-10 Tolliver Anthony
MIN 2010-11 Tolliver A

MIL 2017-18 Wilson D.J.
MIL 2018-19 Wilson D.J.
MIL 2019-20 Wilson D.J.
DEN 2016-17 Beasley Malik
DEN 2017-18 Beasley Malik
DEN 2018-19 Beasley Malik
DEN 2019-20 Beasley Malik
MIN 2019-20 Beasley Malik
MIN 2019-20 Culver Jarrett
GS 2018-19 Evans Jacob
GS 2019-20 Evans Jacob
MIN 2019-20 Evans Jacob
DEN 2016-17 Hernangomez Juan
DEN 2017-18 Hernangomez Juan
DEN 2018-19 Hernangomez Juan
DEN 2019-20 Hernangomez Juan
MIN 2019-20 Hernangomez Juan
CHI 2009-10 Johnson James
CHI 2010-11 Johnson James
TOR 2010-11 Johnson James
TOR 2011-12 Johnson James
SAC 2012-13 Johnson James
MEM 2013-14 Johnson James
TOR 2014-15 Johnson James
TOR 2015-16 Johnson James
MIA 2016-17 Johnson James
MIA 2017-18 Johnson James
MIA 2018-19 Johnson James
MIA 2019-20 Johnson James
MIN 2019-20 Johnson James
POR 2016-17 Layman Jake
POR 2017-18 Layman Jake
POR 2018-19 Layman Jake
MIN 2019-20 Layman Jake
MIN 2019-20 Martin Kelan
MIN 2019-20 McLaughlin Jordan
MIN 2019-20 Nowell Jaylen
MIN 2018-19 Okogie Josh
MIN 2019-20 Okogi

ORL 2018-19 Bamba Mo
ORL 2019-20 Bamba Mo
ORL 2017-18 Birch Khem
ORL 2018-19 Birch Khem
ORL 2019-20 Birch Khem
PHI 2013-14 Carter-Williams Michael
MIL 2014-15 Carter-Williams Michael
PHI 2014-15 Carter-Williams Michael
MIL 2015-16 Carter-Williams Michael
CHI 2016-17 Carter-Williams Michael
CHA 2017-18 Carter-Williams Michael
HOU 2018-19 Carter-Williams Michael
ORL 2018-19 Carter-Williams Michael
ORL 2019-20 Carter-Williams Michael
HOU 2018-19 Clark Gary
HOU 2019-20 Clark Gary
ORL 2019-20 Clark Gary
MIA 2014-15 Ennis III James
NO 2015-16 Ennis III James
MIA 2015-16 Ennis III James
MEM 2015-16 Ennis III James
MEM 2016-17 Ennis III James
DET 2017-18 Ennis III James
MEM 2017-18 Ennis III James
HOU 2018-19 Ennis III James
PHI 2018-19 Ennis III James
ORL 2019-20 Ennis III James
PHI 2019-20 Ennis III James
DEN 2012-13 Fournier Evan
DEN 2013-14 Fournier Evan
ORL 2014-15 Fournier Evan
ORL 2015-16 Fournier Evan
ORL 2016-17 Fournier Evan
ORL 2017-18 Fournier Evan
ORL 2018-19 Fournier Evan
ORL 201

SAC 2018-19 Bagley III Marvin
SAC 2019-20 Bagley III Marvin
GS 2012-13 Barnes Harrison
GS 2013-14 Barnes Harrison
GS 2014-15 Barnes Harrison
GS 2015-16 Barnes Harrison
DAL 2016-17 Barnes Harrison
DAL 2017-18 Barnes Harrison
DAL 2018-19 Barnes Harrison
SAC 2018-19 Barnes Harrison
SAC 2019-20 Barnes Harrison
GS 2012-13 Bazemore Kent
GS 2013-14 Bazemore Kent
LAL 2013-14 Bazemore Kent
ATL 2014-15 Bazemore Kent
ATL 2015-16 Bazemore Kent
ATL 2016-17 Bazemore Kent
ATL 2017-18 Bazemore Kent
ATL 2018-19 Bazemore Kent
POR 2019-20 Bazemore Kent
SAC 2019-20 Bazemore Kent
MIN 2015-16 Bjelica Nemanja
MIN 2016-17 Bjelica Nemanja
MIN 2017-18 Bjelica Nemanja
SAC 2018-19 Bjelica Nemanja
SAC 2019-20 Bjelica Nemanja
SAC 2017-18 Bogdanovic Bogdan
SAC 2018-19 Bogdanovic Bogdan
SAC 2019-20 Bogdanovic Bogdan
DAL 2016-17 Ferrell Yogi
BKN 2016-17 Ferrell Yogi
DAL 2017-18 Ferrell Yogi
SAC 2018-19 Ferrell Yogi
SAC 2019-20 Ferrell Yogi
SAC 2017-18 Fox De'Aaron
SAC 2018-19 Fox De'Aaron
SAC 2019-20 Fox De'Aaron
SAC 

UTAH 2017-18 O'Neale Royce
UTAH 2018-19 O'Neale Royce
UTAH 2019-20 O'Neale Royce
UTAH 2019-20 Oni Miye
UTAH 2019-20 Tucker Rayjon
UTAH 2019-20 Williams-Goss Nigel
UTAH 2019-20 Wright-Foreman Justin
WSH 2012-13 Beal Bradley
WSH 2013-14 Beal Bradley
WSH 2014-15 Beal Bradley
WSH 2015-16 Beal Bradley
WSH 2016-17 Beal Bradley
WSH 2017-18 Beal Bradley
WSH 2018-19 Beal Bradley
WSH 2019-20 Beal Bradley
SA 2016-17 Bertans Davis
SA 2017-18 Bertans Davis
SA 2018-19 Bertans Davis
WSH 2019-20 Bertans Davis
LAL 2018-19 Bonga Isaac
WSH 2019-20 Bonga Isaac
WSH 2018-19 Brown Jr. Troy
WSH 2019-20 Brown Jr. Troy
LAL 2017-18 Bryant Thomas
WSH 2018-19 Bryant Thomas
WSH 2019-20 Bryant Thomas
WSH 2019-20 Hachimura Rui
SA 2007-08 Mahinmi Ian
SA 2009-10 Mahinmi Ian
DAL 2010-11 Mahinmi Ian
DAL 2011-12 Mahinmi Ian
IND 2012-13 Mahinmi Ian
IND 2013-14 Mahinmi Ian
IND 2014-15 Mahinmi Ian
IND 2015-16 Mahinmi Ian
WSH 2016-17 Mahinmi Ian
WSH 2017-18 Mahinmi Ian
WSH 2018-19 Mahinmi Ian
WSH 2019-20 Mahinmi Ian
WSH 2019-

In [7]:
# Close I/O to csv file
fout.close()

### Scratch notes

In [433]:
# Season to process
season = '2020'
# csv out file name
foutName = season+'_players_stats.csv'
# csv file to write cleaned data into
fout = open(foutName, 'w')
# csv writer to write to csv
foutWriter = csv.writer(fout, delimiter=',')

# Specify url
urls = ['https://www.espn.com/nba/stats/player/_/position/point-guard/players/5/table/offensive/sort/avgPoints/dir/desc','https://www.espn.com/nba/stats/player/_/position/point-guard/players/6/table/offensive/sort/avgPoints/dir/desc', 'https://www.espn.com/nba/stats/player/_/position/shooting-guard/players/5/table/offensive/sort/avgPoints/dir/desc', 'https://www.espn.com/nba/stats/player/_/position/shooting-guard/players/6/table/offensive/sort/avgPoints/dir/desc', 'https://www.espn.com/nba/stats/player/_/position/small-forward/players/5/table/offensive/sort/avgPoints/dir/desc', 'https://www.espn.com/nba/stats/player/_/position/small-forward/players/6/table/offensive/sort/avgPoints/dir/desc', 'https://www.espn.com/nba/stats/player/_/position/power-forward/players/5/table/offensive/sort/avgPoints/dir/desc', 'https://www.espn.com/nba/stats/player/_/position/power-forward/players/6/table/offensive/sort/avgPoints/dir/desc', 'https://www.espn.com/nba/stats/player/_/position/center/players/5/table/offensive/sort/avgPoints/dir/desc', 'https://www.espn.com/nba/stats/player/_/position/center/players/6/table/offensive/sort/avgPoints/dir/desc']

In [434]:
def scrape_season_averages (url):
    # Get json response
    page = requests.get(url)
    # Put response into beautifulsoup
    soup = BeautifulSoup(page.text, 'lxml')
    # Find where all player tags are burried
    player_name_tags = soup.find_all('tbody')
    
    # Getting the player's name and team from left side table
    player_list = []
    count = 0
    for tag in player_name_tags[0]:
        # Get player's url extension
        player_url = tag.find('a').get('href')
        player_info = get_player_info(player_url)
        
        # Each tag contains: rank, player name, team in separate subtags
        for subtag in tag:
            # Don't need rank, so skip the rank
            if (count % 2 == 1):
                subtag = subtag.text
                if ((subtag[-2:] == 'NY') or (subtag[-2:] == 'NO') or (subtag[-2:] == 'SA') or (subtag[-2:] == 'GS')):
                    name = subtag[:-2]
                    team = subtag[-2:]
                elif (subtag[-4:] == 'UTAH'):
                    name = subtag[:-4]
                    team = subtag[-4:]
                else:
                    name = subtag[:-3]
                    team = subtag[-3:]                    
                player_list.append([name, team, player_info[0], player_info[1], player_info[2]])
            count += 1
#     print(player_list)

    # Add stats to player's list
    player_counter = 0
    # player_name_tags[1] contains all tags in table 2 (where the actual stats are)
    for player_tag in player_name_tags[1]:
        # Contains all 20 stats
        for stat in player_tag:
            player_list[player_counter].append(stat.text)
        player_counter += 1
#     print(player_list)

    # Write player's stats to line in csv
    for player in player_list:
        line = ''
        for stat in player:
            line += stat
            if (stat != player[len(player)-1]):
                line += ','
        if (player != player_list[len(player_list)-1]):
            line += '\n'
        fout.write(line)

In [436]:
def get_player_info(url):
    # Get json response
    page = requests.get(url)
    # Put response into beautifulsoup
    soup = BeautifulSoup(page.text, 'lxml')
    # Find where all player tags are burried
    stat_tags = soup.find('ul', class_='PlayerHeader__Bio_List flex flex-column list clr-gray-04')

    player_stat = []
    for subtag in stat_tags:
        player_stat.append(subtag.text)

    # print(player_stat[0])
    # Height comes in format: feet' inches"
    # Clean to format as such: feet-inches
    height_weight = player_stat[0].split(' ')
    height = height_weight[0][-2:-1] + '-' + height_weight[1][:1]
    weight = height_weight[2]
    dob = player_stat[1][3:-5]

    return [height, weight, dob]

In [435]:
# Scrape the first table
scrape_season_averages(urls[0])

# Scrape all other tables to existing file foutName
for url in urls[1:]:
    fout.write('\n')
    scrape_season_averages(url)

# # Scrape all other tables to existing file foutName
# with open(foutName, 'a') as fout:
#     for url in urls[1:]:
#         fout.write('\n')
#         scrape_season_averages(url)
    
fout.close()

In [ ]:
def get_player_urls(urls):
    player_url_list = []

    for url in urls:
        # Get json response
        page = requests.get(url)
        # Put response into beautifulsoup
        soup = BeautifulSoup(page.text, 'lxml')
        # Find where all player tags are burried
        player_name_tags = soup.find_all('tbody')
        for tag in player_name_tags[0]:
            player_url = tag.find('a').get('href')
            player_url_list.append(player_url)
    
    return player_url_list

# player_url_list = get_player_urls(urls)
# player_url_list

In [400]:
for url in player_url_list:
    # Get json response
    page = requests.get(url)
    # Put response into beautifulsoup
    soup = BeautifulSoup(page.text, 'lxml')
    # Find where all player tags are burried
    stat_tags = soup.find('ul', class_='PlayerHeader__Bio_List flex flex-column list clr-gray-04')

    player_stat = []
    for subtag in stat_tags:
    #     # Get height and weight
    #     height_weight = stat_tags.find('div', class_='fw-medium clr-black').text
    #     # Height comes in format: feet' inches"
    #     # Clean to format as such: feet-inches
    #     height = height_weight.split(',')[0].split("\'")
    #     height = height[0] + '-' + height[1].strip()[:1]
    #     weight = height_weight.split(',')[1].strip('lbs').strip()
        player_stat.append(subtag.text)

    # print(player_stat[0])
    # Height comes in format: feet' inches"
    # Clean to format as such: feet-inches

    height_weight = player_stat[0].split(' ')

    height = height_weight[0][-2:-1] + '-' + height_weight[1][:1]

    weight = height_weight[2]

    # Get date of birth (DOB)
    dob = player_stat[1][3:-5]

    print(height, weight, dob)

6-1 180 9/19/1998
6-6 200 3/10/1995
6-0 184 5/8/1990
6-5 215 4/6/1993
6-1 190 1/4/1999
6-0 196 3/25/1986
6-2 200 10/4/1988
6-1 195 2/22/1995
6-1 190 3/17/1994
6-1 240 7/20/1996
6-3 190 5/6/1986
6-5 229 12/11/1992
6-1 214 12/9/1989
6-3 220 3/6/1994
6-4 195 2/16/2000
6-1 192 1/26/2000
6-3 209 5/29/1998
6-0 175 7/5/1988
5-1 183 11/10/1987
6-3 188 5/4/1986
6-0 185 9/30/1996
6-3 210 7/25/1989
6-1 190 3/25/1992
6-4 200 7/28/1998
6-3 195 3/26/1994
6-1 180 5/19/1992
6-3 200 9/8/1990
6-2 195 7/15/1990
6-3 200 11/12/1988
6-1 172 9/15/1993
6-4 215 2/23/1997
6-3 174 8/10/1999
6-1 175 5/6/1985
6-3 190 10/21/1990
6-6 190 10/27/1997
6-1 180 8/11/1988
6-4 180 9/19/1996
6-4 190 7/2/1994
6-2 183 6/27/1995
6-3 200 8/1/1992
6-3 180 11/26/1990
6-1 190 8/31/1996
6-1 180 7/12/1988
6-0 196 5/10/1996
6-1 180 2/22/1986
6-3 200 3/5/1996
6-5 185 4/26/1992
6-3 200 8/20/1991
6-3 205 5/4/1998
6-1 200 9/14/1995
6-3 207 6/28/1993
6-6 223 10/24/1996
6-7 205 10/29/1992
6-1 197 2/25/1994
6-2 190 8/3/1995
6-6 206 2/5/1991